In [1]:
# default_exp model
%load_ext lab_black
# nb_black if running in jupyter
%load_ext autoreload
# automatically reload (local) python modules if they are updated
%autoreload 2

In [2]:
# hide
from nbdev.showdoc import *

# Model

    In this notebook you create and test a Python class to wrap your machine learning model.

***input***: toy dataset from data-notebook

***output***: python module containing ML model class 

***description:***

In this notebook you hypothesize, explain and explore machine learning models to solve your problem.

Remember to add `# export` to top of all cells containing functions or classes that you have defined and want to use outside this notebook.

Edit this and the other cells to describe your project.

## Import relevant modules

In [3]:
# export
import numpy as np

# your code here

## Define notebook parameters

Remember, only simple assignments here!

In [4]:
# this cell is tagged with 'parameters'
seed = 0

Make immediate derivations from the parameters:

In [5]:
np.random.seed(seed)

## Import toy data for developing and sw testing your model

In [6]:
# your code here

> Note, that depending on your choice of file format and your variables, you might have to redefine data types once you load data! 

## Explain the math behind

You can use $\LaTeX$ notation for math!

Remember to also explain your choice of loss function and other metrics!

## Explore with simple scripts before constructing the model class

(you can edit out this part before publishing your work)

In [7]:
# your code here

## Define you ML model class

Remember that you can utilize Python class inheritance for complex or alternative model implementations!

In [8]:
# export

# your code here

## Unit test

Unit test your class or classes

In [9]:
# your code here
assert True  #

## Visualize model behaviour with toy data

Runtimes, loss values, metrics, etc.

In [10]:
## your code here

## Output of this notebook

The result of this notebook is a ML model class ready for evaluation with the real data.

You should export classes and functions to `model.py` with `# nbdev_build_lib` (workflows will do this automatically).

## You can move on to loss notebook!